In [ ]:
from collections import namedtuple
from random import choice
from tqdm.auto import tqdm
import numpy as np
from queue import PriorityQueue

In [ ]:
PUZZLE_DIM = 7  
RANDOMIZE_STEPS = 150

def available_actions(state: np.ndarray) -> list:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(((x, y), (x - 1, y))) 
    if x < PUZZLE_DIM - 1:
        actions.append(((x, y), (x + 1, y)))  
    if y > 0:
        actions.append(((x, y), (x, y - 1))) 
    if y < PUZZLE_DIM - 1:
        actions.append(((x, y), (x, y + 1))) 
    return actions

def do_action(state: np.ndarray, action: tuple) -> np.ndarray:
    new_state = state.copy()
    (x1, y1), (x2, y2) = action
    new_state[x1, y1], new_state[x2, y2] = new_state[x2, y2], new_state[x1, y1]
    return new_state

def optimized_heuristic(state: np.ndarray, goal: np.ndarray) -> int:
    """ Ottimizzazione dell'euristica basata sulla distanza di Manhattan e il numero di tessere fuori posto. """
    manhattan_distance = 0
    misplacement_count = 0
    for x in range(PUZZLE_DIM):
        for y in range(PUZZLE_DIM):
            value = state[x, y]
            if value != 0:  
                goal_x, goal_y = divmod(value - 1, PUZZLE_DIM)
                manhattan_distance += abs(x - goal_x) + abs(y - goal_y)
                if goal[x, y] != value:
                    misplacement_count += 1

    return manhattan_distance + 2 * misplacement_count  

def solve_puzzle(start_state: np.ndarray, goal_state: np.ndarray):
    """ Risolvi il puzzle n²-1 usando A* con coda di priorità. """
    frontier = PriorityQueue()
    frontier.put((0, start_state.tolist(), [start_state.tolist()], 0)) 
    visited = set()

    while not frontier.empty():
        f_cost, current_state, path, g_cost = frontier.get()
        current_state = np.array(current_state)
        if np.array_equal(current_state, goal_state):
            return path  

        state_tuple = tuple(map(tuple, current_state))
        if state_tuple in visited:
            continue
        visited.add(state_tuple)

        for action in available_actions(current_state):
            new_state = do_action(current_state, action)
            new_g_cost = g_cost + 1
            new_f_cost = new_g_cost + optimized_heuristic(new_state, goal_state)
            new_path = path + [new_state.tolist()]
            frontier.put((new_f_cost, new_state.tolist(), new_path, new_g_cost))

    return None  


In [ ]:
if PUZZLE_DIM == 0:
    print("Soluzione trovata in 0 mosse!")
    print("Passo 0:")
    print([])
elif PUZZLE_DIM == 1:
    state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
    print("Soluzione trovata in 0 mosse!")
    print("Passo 0:")
    print(state)  
else:
    state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))

    for _ in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
        state = do_action(state, choice(available_actions(state)))

    goal_state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
    solution = solve_puzzle(state, goal_state)

    if solution:
        print(f"Soluzione trovata in {len(solution) - 1} mosse!") 
        for step, s in enumerate(solution):
            print(f"Passo {step}:")
            print(np.array(s))  
    else:
        print("Nessuna soluzione trovata.")